##I. Download and Setup


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/...
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install cuda gcc-5 g++-5 -y -qq;
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt install cuda;

In [ ]:
!pip install dgl
!pip install category_encoders
!pip install dgl-cu116 dglgo -f https://data.dgl.ai/wheels/repo.html

In [ ]:
import dgl.nn as dglnn
from dgl import from_networkx
import torch.nn as nn
import torch as th
import torch.nn.functional as F
import dgl.function as fn
import networkx as nx
import pandas as pd
import socket
import struct
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/NF-CSE-CIC-IDS2018/NF-CSE-CIC-IDS2018.csv')

In [ ]:
# data_test = pd.read_csv('/content/drive/MyDrive/NF-UNSW-NB15/NF-UNSW-NB15.csv')

In [ ]:
data

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
0,172.31.66.17,51128,23.36.69.189,443,6,91.000,152,0,3,0,194,4285680,0,Benign
1,34.195.59.178,443,172.31.66.17,51036,6,91.178,994,979,7,7,24,4234714,0,Benign
2,112.215.171.212,12262,172.31.66.17,445,6,0.000,585,344,5,4,30,4294106,0,Benign
3,172.31.66.17,61023,172.31.0.2,53,17,0.000,136,168,2,2,0,4293985,0,Benign
4,34.204.112.168,443,172.31.66.17,51037,6,91.178,72,40,1,1,24,4294845,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8392396,172.31.69.7,22,122.226.181.164,40810,6,0.000,2601,0,12,0,27,0,0,Benign
8392397,45.70.167.112,15476,172.31.69.7,23,6,0.000,44,0,1,0,2,0,0,Benign
8392398,172.31.69.7,23,45.70.167.112,15476,6,0.000,40,0,1,0,20,0,0,Benign
8392399,172.31.69.7,56407,172.31.0.2,53,17,0.000,72,0,1,0,0,0,0,Benign


In [ ]:
data.columns

Index(['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT',
       'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 'IN_PKTS', 'OUT_PKTS',
       'TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS', 'Label', 'Attack'],
      dtype='object')

In [ ]:
data.Attack.value_counts()

Benign                      7373198
DDoS attacks-LOIC-HTTP       378199
FTP-BruteForce               193360
DoS attacks-Hulk             108136
DoS attacks-SlowHTTPTest     105550
SSH-Bruteforce                94237
Infilteration                 62072
DoS attacks-GoldenEye         32850
DoS attacks-Slowloris         22825
Bot                           15683
Brute Force -Web               2613
Brute Force -XSS               1745
DDOS attack-LOIC-UDP           1667
DDOS attack-HOIC                230
SQL Injection                    36
Name: Attack, dtype: int64

In [ ]:
def change_label(data):
  data.Attack.replace(['DDoS attacks-LOIC-HTTP', 'DoS attacks-Hulk', 'DoS attacks-SlowHTTPTest', 'DoS attacks-GoldenEye', 
                    'DoS attacks-Slowloris', 'DDOS attack-LOIC-UDP', 'DDOS attack-HOIC'],'Dos',inplace=True)
  data.Attack.replace(['FTP-BruteForce', 'SSH-Bruteforce', 'Brute Force -Web'],'Probe',inplace=True)
  data.Attack.replace(['SQL Injection', 'Brute Force -XSS'],'U2R',inplace=True)
  data.Attack.replace(['Benign'],'Normal',inplace=True)
  data.Attack.replace(['Bot', 'Infilteration'],'R2L',inplace=True)
change_label(data)
data.Attack.value_counts()

Normal    7373198
Dos        649457
Probe      290210
R2L         77755
U2R          1781
Name: Attack, dtype: int64

In [ ]:
# def change_label(data):
#   data.Attack.replace(['DoS'],'Dos',inplace=True)
#   data.Attack.replace(['Reconnaissance'],'Probe',inplace=True)
#   data.Attack.replace(['Generic','Analysis','Exploits','Shellcode','Fuzzers'],'U2R',inplace=True)
#   data.Attack.replace(['Benign'],'Normal',inplace=True)
#   data.Attack.replace(['Backdoor','Worms'],'R2L',inplace=True)
# change_label(data_test)
# data_test.Attack.value_counts()

In [ ]:
data.drop(data.loc[data['Attack']=='Normal'].index[:6900000], axis=0, inplace=True)
data.drop(data.loc[data['Attack']=='Dos'].index[:520000], axis=0, inplace=True)
data.drop(data.loc[data['Attack']=='Probe'].index[:100000], axis=0, inplace=True)
data.drop(data.loc[data['Attack']=='R2L'].index[:30000], axis=0, inplace=True)

In [ ]:
# data_test.drop(data_test.loc[data_test['Attack']=='Normal'].index[:1000000], axis=0, inplace=True)

In [ ]:
data.Attack.value_counts()

Normal    473198
Probe     190210
Dos       129457
R2L        47755
U2R         1781
Name: Attack, dtype: int64

In [ ]:
# data_test.Attack.value_counts()

In [ ]:
data.head()

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
1917040,52.14.77.172,1024,172.31.69.28,500,17,79.178,528,0,1,0,0,0,1,U2R
1917052,13.58.42.57,1028,172.31.69.28,500,17,79.178,2640,0,5,0,0,4247475,1,U2R
1917058,13.58.42.57,1028,172.31.69.28,500,17,79.178,528,0,1,0,0,0,1,U2R
1917061,52.14.77.172,1024,172.31.69.28,500,17,79.178,2640,0,5,0,0,4247475,1,U2R
1917066,52.14.77.172,1024,172.31.69.28,500,17,79.178,528,0,1,0,0,0,1,U2R


In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(", ".join(result))
        
def analyze(df):
    print()
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,\
                int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])
analyze(data)


842401 rows
** IPV4_SRC_ADDR:6891 (0%)
** L4_SRC_PORT:30729 (3%)
** IPV4_DST_ADDR:6806 (0%)
** L4_DST_PORT:30981 (3%)
** PROTOCOL:[6:92.56%, 17:7.16%, 1:0.27%, 2:0.0%, 58:0.0%, 0:0.0%]
** L7_PROTO:[0.0:97.61%, 5.126:0.47%, 91.126:0.38%, 91.178:0.22%, 79.178:0.19%, 91.125:0.13%, 7.0:0.12%, 5.178:0.11%, 41.0:0.07%, 91.0:0.07%, 5.119:0.07%, 5.24:0.06%, 91.119:0.05%, 127.0:0.05%, 91.22:0.04%, 7.126:0.04%, 5.239:0.03%, 91.12:0.03%, 5.233:0.03%, 5.2120000000000015:0.02%, 88.0:0.02%, 92.0:0.02%, 5.12:0.02%, 10.0:0.01%, 91.212:0.01%, 91.24:0.01%, 91.239:0.01%, 7.178:0.01%, 5.14:0.01%, 5.202999999999999:0.01%, 5.124:0.01%, 5.7:0.01%, 91.14:0.01%, 77.0:0.01%, 91.233:0.0%, 91.7:0.0%, 114.0:0.0%, 91.221:0.0%, 5.215:0.0%, 5.231:0.0%, 7.22:0.0%, 238.0:0.0%, 5.68:0.0%, 89.0:0.0%, 131.0:0.0%, 91.215:0.0%, 29.0:0.0%, 5.218999999999999:0.0%, 170.0:0.0%, 100.0:0.0%, 158.0:0.0%, 161.0:0.0%, 13.0:0.0%, 20.0:0.0%, 91.68:0.0%, 51.0:0.0%, 7.2189999999999985:0.0%, 4.0:0.0%, 7.125:0.0%, 1.0:0.0%, 244.0:0.0%, 7

In [ ]:
data['IPV4_SRC_ADDR'] = data.IPV4_SRC_ADDR.apply(lambda x: socket.inet_ntoa(struct.pack('>I', random.randint(0xac100001, 0xac1f0001))))

#test
# data_test['IPV4_SRC_ADDR'] = data_test.IPV4_SRC_ADDR.apply(lambda x: socket.inet_ntoa(struct.pack('>I', random.randint(0xac100001, 0xac1f0001))))

In [ ]:
data['IPV4_SRC_ADDR'] = data.IPV4_SRC_ADDR.apply(str)
data['L4_SRC_PORT'] = data.L4_SRC_PORT.apply(str)
data['IPV4_DST_ADDR'] = data.IPV4_DST_ADDR.apply(str)
data['L4_DST_PORT'] = data.L4_DST_PORT.apply(str)

#test
# data_test['IPV4_SRC_ADDR'] = data_test.IPV4_SRC_ADDR.apply(str)
# data_test['L4_SRC_PORT'] = data_test.L4_SRC_PORT.apply(str)
# data_test['IPV4_DST_ADDR'] = data_test.IPV4_DST_ADDR.apply(str)
# data_test['L4_DST_PORT'] = data_test.L4_DST_PORT.apply(str)

In [ ]:
data['IPV4_SRC_ADDR'] = data['IPV4_SRC_ADDR'] + ':' + data['L4_SRC_PORT']
data['IPV4_DST_ADDR'] = data['IPV4_DST_ADDR'] + ':' + data['L4_DST_PORT']

#test
# data_test['IPV4_SRC_ADDR'] = data_test['IPV4_SRC_ADDR'] + ':' + data_test['L4_SRC_PORT']
# data_test['IPV4_DST_ADDR'] = data_test['IPV4_DST_ADDR'] + ':' + data_test['L4_DST_PORT']

In [ ]:
data.drop(columns=['L4_SRC_PORT','L4_DST_PORT'],inplace=True)

#test
# data_test.drop(columns=['L4_SRC_PORT','L4_DST_PORT'],inplace=True)

In [ ]:
data.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,Label,Attack
1917040,172.22.161.178:1024,172.31.69.28:500,17,79.178,528,0,1,0,0,0,1,U2R
1917052,172.25.235.141:1028,172.31.69.28:500,17,79.178,2640,0,5,0,0,4247475,1,U2R
1917058,172.16.233.204:1028,172.31.69.28:500,17,79.178,528,0,1,0,0,0,1,U2R
1917061,172.30.231.150:1024,172.31.69.28:500,17,79.178,2640,0,5,0,0,4247475,1,U2R
1917066,172.22.79.108:1024,172.31.69.28:500,17,79.178,528,0,1,0,0,0,1,U2R


In [ ]:
data.drop(columns=['Label'],inplace = True)

#test
# data_test.drop(columns=['Label'],inplace = True)

In [ ]:
data.rename(columns={"Attack": "label"},inplace = True)

#test
# data_test.rename(columns={"Attack": "label_test"},inplace = True)

In [ ]:
le = LabelEncoder()
le.fit_transform(data.label.values)
data['label'] = le.transform(data['label'])

#test
# le = LabelEncoder()
# le.fit_transform(data_test.label_test.values)
# data_test['label_test'] = le.transform(data_test['label_test'])

In [ ]:
label = data.label

#test
# label_test = data_test.label_test

In [ ]:
data.drop(columns=['label'],inplace = True)

#test
# data_test.drop(columns=['label_test'],inplace = True)

In [ ]:
scaler = StandardScaler()

In [ ]:
data =  pd.concat([data, label], axis=1)

#test
# data_test =  pd.concat([data_test, label_test], axis=1)

In [ ]:
data.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,label
1917040,172.22.161.178:1024,172.31.69.28:500,17,79.178,528,0,1,0,0,0,4
1917052,172.25.235.141:1028,172.31.69.28:500,17,79.178,2640,0,5,0,0,4247475,4
1917058,172.16.233.204:1028,172.31.69.28:500,17,79.178,528,0,1,0,0,0,4
1917061,172.30.231.150:1024,172.31.69.28:500,17,79.178,2640,0,5,0,0,4247475,4
1917066,172.22.79.108:1024,172.31.69.28:500,17,79.178,528,0,1,0,0,0,4


In [ ]:
# data_test.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
     data, label, test_size=0.2, random_state=123,stratify= label)

In [ ]:
# X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(
#      data_test, label_test, test_size=0.2, random_state=123,stratify= label_test)

In [ ]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL'])
encoder.fit(X_train, y_train)
X_train = encoder.transform(X_train)

/usr/local/lib/python3.8/dist-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/usr/local/lib/python3.8/dist-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [ ]:
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns ))  - set(list(['label'])) )
X_train[cols_to_norm] = scaler.fit_transform(X_train[cols_to_norm])

In [ ]:
X_train['h'] = X_train[ cols_to_norm ].values.tolist()

In [ ]:
X_train

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,OUT_BYTES,IN_PKTS,OUT_PKTS,TCP_FLAGS,FLOW_DURATION_MILLISECONDS,label,h
7694387,172.16.56.53:49610,172.31.69.25:80,-0.274970,-0.154754,-0.064988,-0.021221,-0.051133,-0.056753,-1.564261,-0.247294,0,"[-0.06498760235928015, -1.5642605061199397, -0..."
8168378,172.20.1.44:57588,172.31.69.25:21,-0.274970,-0.154754,-0.066765,-0.021221,-0.051783,-0.056753,1.391509,-0.247294,2,"[-0.066764835526842, 1.391508693481254, -0.056..."
2200952,172.19.228.159:52510,172.31.0.2:53,3.599184,-0.154754,-0.066580,0.037395,-0.051621,0.394235,1.198502,4.050131,3,"[-0.06658031405042847, 1.1985017770937283, 0.3..."
4361491,172.25.212.86:61239,172.31.69.8:3389,-0.274970,-0.154754,-0.063327,0.294264,-0.050646,1.521704,-1.887145,4.048492,3,"[-0.06332690907155843, -1.8871449122908812, 1...."
7795570,172.24.92.30:50794,172.31.69.18:20766,-0.274970,-0.154754,-0.066813,-0.021221,-0.051783,-0.056753,1.391509,-0.247294,1,"[-0.06681339381010871, 1.391508693481254, -0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...
8116640,172.16.130.75:21,18.221.219.4:34144,-0.274970,-0.154754,-0.066813,-0.021221,-0.051783,-0.056753,-0.563951,-0.247294,1,"[-0.06681339381010871, -0.5639507958420437, -0..."
7769227,172.27.166.193:80,18.217.165.70:43984,-0.274970,-0.154754,-0.066765,-0.021221,-0.051783,-0.056753,-0.337624,-0.247294,1,"[-0.066764835526842, -0.33762446468591084, -0...."
7777331,172.18.157.247:46674,172.31.69.25:80,-0.274970,-0.154754,-0.066100,-0.021221,-0.051621,-0.056753,-0.994701,-0.247294,0,"[-0.06609958704608797, -0.9947008146029278, -0..."
8106451,172.16.6.237:52202,172.31.69.25:21,-0.274970,-0.154754,-0.066765,-0.021221,-0.051783,-0.056753,1.391509,-0.247294,2,"[-0.066764835526842, 1.391508693481254, -0.056..."


In [ ]:
X_train.label.value_counts()

1    378558
2    152168
0    103565
3     38204
4      1425
Name: label, dtype: int64

In [ ]:
G = nx.from_pandas_edgelist(X_train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR", ['h','label'],create_using=nx.MultiGraph())

In [ ]:
G = G.to_directed()


In [ ]:
G

In [ ]:
G = from_networkx(G,edge_attrs=['h','label'] )


In [ ]:
# Eq1
G.ndata['h'] = th.ones(G.num_nodes(), G.edata['h'].shape[1])

In [ ]:
G.ndata['h'].shape

torch.Size([753488, 8])

In [ ]:
G.ndata['h'].shape

torch.Size([753488, 8])

In [ ]:
G.edata['train_mask'] = th.ones(len(G.edata['h']), dtype=th.bool)

In [ ]:
G.edata['train_mask']

tensor([True, True, True,  ..., True, True, True])

In [ ]:
G.edata['train_mask'].shape 

torch.Size([1347840])

In [ ]:
G.ndata['h'] = th.reshape(G.ndata['h'], (G.ndata['h'].shape[0], 1,G.ndata['h'].shape[1]))
G.edata['h'] = th.reshape(G.edata['h'], (G.edata['h'].shape[0], 1,G.edata['h'].shape[1]))

In [ ]:
def compute_accuracy(pred, labels):
    return (pred.argmax(1) == labels).float().mean().item()

In [ ]:
class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
        super(SAGELayer, self).__init__()
        ### force to output fix dimensions
        self.W_msg = nn.Linear(ndim_in + edims, ndim_out)
        ### apply weight
        self.W_apply = nn.Linear(ndim_in + ndim_out, ndim_out)
        self.activation = activation

    def message_func(self, edges):
        return {'m': self.W_msg(th.cat([edges.src['h'], edges.data['h']], 2))}

    def forward(self, g_dgl, nfeats, efeats):
        with g_dgl.local_scope():
            g = g_dgl
            g.ndata['h'] = nfeats
            g.edata['h'] = efeats
            # Eq4
            g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
            # Eq5          
            g.ndata['h'] = F.relu(self.W_apply(th.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))
            return g.ndata['h']


class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation, dropout):
        super(SAGE, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(SAGELayer(ndim_in, edim, 128, activation))
        self.layers.append(SAGELayer(128, edim, ndim_out, activation))
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, g, nfeats, efeats):
        for i, layer in enumerate(self.layers):
            if i != 0:
                nfeats = self.dropout(nfeats)
            nfeats = layer(g, nfeats, efeats)
        return nfeats.sum(1)

In [ ]:
class MLPPredictor(nn.Module):
    def __init__(self, in_features, out_classes):
        super().__init__()
        self.W = nn.Linear(in_features * 2, out_classes)

    def apply_edges(self, edges):
        h_u = edges.src['h']
        h_v = edges.dst['h']
        global score
        global emb
        emb = th.cat([h_u, h_v], 1)
        score = self.W(th.cat([h_u, h_v], 1))
        return {'score': score}

    def forward(self, graph, h):
        with graph.local_scope():
            graph.ndata['h'] = h
            graph.apply_edges(self.apply_edges)
            return graph.edata['score']

In [ ]:
class Model(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation, dropout):
        super().__init__()
        self.gnn = SAGE(ndim_in, ndim_out, edim, activation, dropout)
        self.pred = MLPPredictor(ndim_out, 5)
    def forward(self, g, nfeats, efeats):
        h = self.gnn(g, nfeats, efeats)
        return self.pred(g, h)

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes= np.unique(G.edata['label'].cpu().numpy()),
                                                 y= G.edata['label'].cpu().numpy())

In [ ]:
class_weights = th.FloatTensor(class_weights).cuda()
criterion = nn.CrossEntropyLoss(weight = class_weights)

RuntimeError: ignored

In [ ]:
G = G.to('cuda:0')
G.device

In [ ]:
G.ndata['h'].device


In [ ]:
G.edata['h'].device


In [ ]:
G.ndata['h'].shape

In [ ]:
G.edata['h'].shape

In [ ]:
node_features = G.ndata['h']
edge_features = G.edata['h']

edge_label = G.edata['label']
train_mask = G.edata['train_mask']

model = Model(G.ndata['h'].shape[2], 128, G.ndata['h'].shape[2], F.relu, 0.2).cuda()
opt = th.optim.Adam(model.parameters())

for epoch in range(1,2000):
    pred = model(G, node_features,edge_features).cuda()
    loss = criterion(pred[train_mask] ,edge_label[train_mask])
    opt.zero_grad()
    loss.backward()
    opt.step()
    if epoch % 100 == 0:
      print('Epoch:', epoch,'Training acc:', compute_accuracy(pred[train_mask], edge_label[train_mask]))


In [ ]:
model

In [ ]:
X_test = encoder.transform(X_test)

#test
# X_test_t = encoder.transform(X_test_t)

In [ ]:
#test
# cols_to_norm_test = list(set(list(X_train_t.iloc[:, 2:].columns ))  - set(list(['label_test'])) )

X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
# X_test_t[cols_to_norm_test] = scaler.transform(X_test_t[cols_to_norm_test])

In [ ]:
X_test
# X_test_t

In [ ]:
X_test.label.value_counts()
# X_test_t.label_test.value_counts()

In [ ]:
X_test['h'] = X_test[ cols_to_norm ].values.tolist()

In [ ]:
# X_test_t['h'] = X_test_t[ cols_to_norm_test ].values.tolist()

In [ ]:
G_test = nx.from_pandas_edgelist(X_test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR", ['h','label'],create_using=nx.MultiGraph())
G_test = G_test.to_directed()
G_test = from_networkx(G_test,edge_attrs=['h','label'] )
actual = G_test.edata.pop('label')
G_test.ndata['feature'] = th.ones(G_test.num_nodes(), G.ndata['h'].shape[2])

In [ ]:
# G_test = nx.from_pandas_edgelist(X_test_t, "IPV4_SRC_ADDR", "IPV4_DST_ADDR", ['h','label_test'],create_using=nx.MultiGraph())
# G_test = G_test.to_directed()
# G_test = from_networkx(G_test,edge_attrs=['h','label_test'] )
# actual = G_test.edata.pop('label_test')
# G_test.ndata['feature'] = th.ones(G_test.num_nodes(), G.ndata['h'].shape[2])

In [ ]:
G_test.ndata['feature'] = th.reshape(G_test.ndata['feature'], (G_test.ndata['feature'].shape[0], 1, G_test.ndata['feature'].shape[1]))

In [ ]:
G_test.edata['h'] = th.reshape(G_test.edata['h'], (G_test.edata['h'].shape[0], 1, G_test.edata['h'].shape[1]))


In [ ]:
G_test = G_test.to('cuda:0')


In [ ]:
import timeit
start_time = timeit.default_timer()
node_features_test = G_test.ndata['feature']
edge_features_test = G_test.edata['h']
test_pred = model(G_test, node_features_test, edge_features_test).cuda()
elapsed = timeit.default_timer() - start_time

In [ ]:
print(str(elapsed) + ' seconds')


In [ ]:
test_pred = test_pred.argmax(1)


In [ ]:
test_pred = th.Tensor.cpu(test_pred).detach().numpy()


In [ ]:
actual = le.inverse_transform(actual)
test_pred = le.inverse_transform(test_pred)

In [ ]:
from sklearn.metrics import plot_confusion_matrix


In [ ]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(12, 12))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

plot_confusion_matrix(cm = confusion_matrix(actual, test_pred), 
                      normalize    = False,
                      target_names = np.unique(actual),
                      title        = "Confusion Matrix")

In [ ]:
from sklearn.metrics import classification_report
target_names = np.unique(actual)
print(classification_report(actual, test_pred, target_names=target_names, digits=4))

In [ ]:
actual

In [ ]:
test_pred

In [ ]:
# # ROC curve：

# from tensorflow.keras.utils import to_categorical
# from sklearn.metrics import roc_curve, auc
# from sklearn.metrics import roc_auc_score
# from itertools import cycle
# def RoC_Curve(y_score, y, labels, title): 
#     y_cat = to_categorical(y)

        
#     # Compute ROC curve and ROC area for each class
#     fpr = dict()
#     tpr = dict()
#     roc_auc = dict()
#     lw = 2
#     # First aggregate all false positive rates
#     n_classes = len(labels)
#     print('n_classes:', n_classes)

#     for i in range(n_classes):
#         fpr[i], tpr[i], _ = roc_curve(y_cat[:, i], y_score[:, i])
#         roc_auc[i] = auc(fpr[i], tpr[i])

        
        
        
#     # Compute micro-average ROC curve and ROC area
#     fpr["micro"], tpr["micro"], _ = roc_curve(y_cat.ravel(), y_score.ravel())
#     roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

#     # First aggregate all false positive rates
#     all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

#     # Then interpolate all ROC curves at this points
#     mean_tpr = np.zeros_like(all_fpr)
#     for i in range(n_classes):
#         mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

        
        
        
#     # Finally average it and compute AUC
#     mean_tpr /= n_classes

#     fpr["macro"] = all_fpr
#     tpr["macro"] = mean_tpr
#     roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

#     # Plot all ROC curves
#     plt.figure(figsize=(8,8))
#     plt.plot(fpr["micro"], tpr["micro"],
#              label='micro-average ROC curve (area = {0:0.4f})'
#                    ''.format(roc_auc["micro"]),
#              color='deeppink', linestyle=':', linewidth=4)

#     plt.plot(fpr["macro"], tpr["macro"],
#              label='macro-average ROC curve (area = {0:0.4f})'
#                    ''.format(roc_auc["macro"]),
#              color='navy', linestyle=':', linewidth=4)

#     colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
#     for i, color in zip(range(n_classes), colors):
#         plt.plot(fpr[i], tpr[i], color=color, lw=lw,
#                  label=f'ROC curve of class {labels[i]} (area = {roc_auc[i]:0.4f})')

#     plt.plot([0, 1], [0, 1], 'k--', lw=lw)
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title(title, fontsize=16)
#     plt.legend(loc="lower right")
#     plt.show()


# y_eval= le.fit_transform(actual)
# y_test= le.fit_transform(test_pred)
# RoC_Curve(y_test, y_eval, le.classes_, title=' 0:Dos  1:normal  2:Probe  3:R2L  4:U2L \n\n ROC for E-GraphSAGE')

In [ ]:
cm = confusion_matrix(actual, test_pred)

FP = cm.sum(axis=0) - np.diag(cm)  
FN = cm.sum(axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = cm.sum() - (FP + FN + TP)

FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

print("TP = ", TP)
print("TN = ", TN)
print("FP = ", FP)
print("FN = ", FN)
print("\n")

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
print("TPR = ", TPR , "  True positive rate, Sensitivity, hit rate, or recall")
# Fall out or false positive rate
FPR = FP/(FP+TN)
print("FPR = ", FPR, "  False positive rate or fall out")
# Specificity or true negative rate
TNR = TN/(TN+FP)
print("TNR = ", TNR, "  True negative rate or specificity")
# False negative rate
FNR = FN/(TP+FN)
print("FNR = ", FNR, "  False negative rate")
print("\n")

# Precision or positive predictive value
PPV = TP/(TP+FP)
print("PPV = ", PPV, "  Positive predictive value or precision")
# Negative predictive value
NPV = TN/(TN+FN)
print("NPV = ", NPV, "  Negative predictive value")
# False discovery rate
FDR = FP/(TP+FP)
print("FDR = ", FDR, "  False discovery rate")
print("\n")

#Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print(target_names)
print("ACC = ", ACC)